In [1]:
import numpy as np
import matplotlib.pyplot as plt
from abtem import *
from abtem.utils import GaussianDistribution
from ase.io import read
from ase.build import surface, bulk
from tqdm.auto import tqdm
from ase.build import graphene

from tqdm.auto import tqdm
from abtem.measure import stack_measurements
from abtem.structures import orthogonalize_cell
from abtem.measure import center_of_mass
from ase.spacegroup import crystal

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:60% !important; }</style>"))

In [29]:
atoms = graphene()
atoms = orthogonalize_cell(atoms)
atoms *= (3,2,1)

atoms.center(vacuum=2, axis=2)

probe = SMatrix(energy=80e3, semiangle_cutoff=30, device='cpu')
potential = Potential(atoms, #FrozenPhonons(atoms, sigmas=.06, num_configs=32), 
                      sampling=.04, 
                      projection='infinite',
                      parametrization='kirkland')
probe.grid.match(potential)

S = probe.multislice(potential, pbar=True)
S = S.downsample()

detector = PixelatedDetector(max_angle=150)

distribution = GaussianDistribution(0, 50, 7)
scan = GridScan((0, 0), (potential.extent[0] / 3, potential.extent[1] / 2), sampling = probe.ctf.nyquist_sampling * .75)

graphene_measurements = []
for defocus in tqdm(distribution.samples, desc='Defocus'):
    S.ctf.defocus = defocus
    graphene_measurements.append(S.scan(scan, detector, pbar=False)) 

array = np.stack([measurement.array for measurement in graphene_measurements])
measurement = Measurement(array, calibrations=(None,) + graphene_measurements[0].calibrations)
measurement.write('data/graphene_defocus_series.hdf5')

Multislice:   0%|          | 0/8 [00:00<?, ?it/s]

Defocus:   0%|          | 0/7 [00:00<?, ?it/s]

'data/graphene_defocus_series.hdf5'

In [20]:
measurement.array.shape

(7, 15, 25, 55, 63)

In [37]:
atom_pos = [(0.0, 0.0, 0.0), (0.5, 0.5, 0.5), (0.5, 0.5, 0.0)]
atoms = crystal(['Sr','Ti','O'], atom_pos, spacegroup=221, cellpar=3.905, size=(1, 1, 1))

atoms *= (2, 2, 10)
atoms.center()

probe = SMatrix(energy=120e3, semiangle_cutoff=30, device='cpu')
potential = Potential(FrozenPhonons(atoms, sigmas=.0, num_configs=16), 
                      sampling=.04, 
                      projection='infinite',
                      slice_thickness=2,
                      parametrization='kirkland')
probe.grid.match(potential)

S = probe.multislice(potential, pbar=True)
S = S.downsample()

distribution = GaussianDistribution(0, 50, 7)
scan = GridScan((0, 0), (potential.extent[0] / 2, potential.extent[1] / 2), sampling = probe.ctf.nyquist_sampling * .75)

detector = PixelatedDetector(max_angle=150)

srtio3_measurements = []
for defocus in tqdm(distribution.samples, desc='Defocus'):
    S.ctf.defocus = defocus
    srtio3_measurements.append(S.scan(scan, detector, pbar=False))
    
array = np.stack([measurement.array for measurement in srtio3_measurements])
measurement = Measurement(array, calibrations=(None,) + srtio3_measurements[0].calibrations)
measurement.write('data/srtio3_defocus_series.hdf5')

Multislice:   0%|          | 0/20 [00:00<?, ?it/s]

Defocus:   0%|          | 0/7 [00:00<?, ?it/s]

'data/srtio3_defocus_series.hdf5'

In [25]:
measurement.array.shape

(7, 13, 13, 71, 71)

In [38]:
graphene_measurement = Measurement.read('data/graphene_defocus_series.hdf5')
srtio3_measurement = Measurement.read('data/srtio3_defocus_series.hdf5')
distribution = GaussianDistribution(0, 50, 7)
samples = distribution.samples

In [42]:
from abtem.visualize.interactive.canvas import Canvas
from abtem.visualize.interactive.artists import MeasurementArtist2d
from abtem.visualize.interactive.tools import SelectPixelTool
from abtem.measure import block_zeroth_order_spot, bandlimit
import functools
import ipywidgets as widgets
from traitlets import link

canvas1 = Canvas()
canvas2 = Canvas()

image_artist1 = MeasurementArtist2d()
image_artist2 = MeasurementArtist2d()
canvas1.artists = {'image_artist': image_artist1}
canvas2.artists = {'image_artist': image_artist2}

position_tool = SelectPixelTool(image_artist=image_artist1.image_artist, marker=True)

energy_spread_slider = widgets.FloatSlider(description='Energy spread', min=1, max=50, value=30, step=1, continuous_update=False)
source_size_slider = widgets.FloatSlider(description='Source size', min=.01, max=.6, value=.3, step=.01, continuous_update=False)
power_scale_slider = widgets.FloatSlider(description='Power scale', min=.01, max=1, value=1, step=.01, continuous_update=False)
autoadjust_button = widgets.ToggleButton(description='Autoadjust colorscale', value=True)
structure_dropdown = widgets.Dropdown(description='Structure', options=['Graphene', 'SrTiO3'], value='Graphene')
signal_dropdown = widgets.Dropdown(description='Signal', options=['HAADF', 'BF', 'COM'], value='HAADF')

block_direct_button = widgets.ToggleButton(description='Block direct beam')
link((autoadjust_button, 'value'), (image_artist1.image_artist, 'autoadjust_colorscale'))

@functools.lru_cache(maxsize=1)
def new_4d_measurement(structure, energy_spread, source_size):
    if structure == 'Graphene':
        calibrations = graphene_measurement[0].calibrations
        array = graphene_measurement.array
    else:
        calibrations = srtio3_measurement[0].calibrations
        array = srtio3_measurement.array
    
    values = np.exp(- samples ** 2 / (2 * energy_spread_slider.value ** 2))
    values /= values.sum()
    measurement = Measurement((array * values[(slice(None),) + (None,)*4]).sum(0), calibrations=calibrations)
    return measurement.gaussian_filter((source_size_slider.value,) * 2 + (0.,) * 2)


def update_image(*args):
    measurement = new_4d_measurement(structure_dropdown.value, energy_spread_slider.value, source_size_slider.value)
    
    if structure_dropdown.value == 'Graphene':
        reps = (3,2)
        semiangle_cutoff = 30
    else:
        reps = (4,4)
        semiangle_cutoff = 20
    
    if signal_dropdown.value == 'HAADF':
        integrated_measurement = AnnularDetector(inner=50, outer=150).integrate(measurement)
    elif signal_dropdown.value == 'BF':
        integrated_measurement = AnnularDetector(inner=0, outer=30).integrate(measurement)
    elif signal_dropdown.value == 'COM':
        measurement = bandlimit(measurement, cutoff=80)
        integrated_measurement = center_of_mass(measurement, return_magnitude=True)

    image_artist1.measurement = integrated_measurement.interpolate(.1).tile(reps)
    
    update_diffraction_pattern()
    canvas1.adjust_limits_to_artists()

def update_diffraction_pattern(*args):
    measurement = new_4d_measurement(structure_dropdown.value, energy_spread_slider.value, source_size_slider.value).copy()
    index_x = position_tool.indices[0] % measurement.shape[0]
    index_y = position_tool.indices[1] % measurement.shape[1]
    
    if structure_dropdown.value == 'Graphene':
        semiangle_cutoff = 30
    else:
        semiangle_cutoff = 20
    
    if block_direct_button.value:
        measurement = block_zeroth_order_spot(measurement, semiangle_cutoff)
    
    index_measurement = measurement[index_x, index_y]
    
    index_measurement._array = index_measurement._array ** power_scale_slider.value
    
    image_artist2.measurement = index_measurement
    
    
energy_spread_slider.observe(update_image, 'value')
autoadjust_button.observe(update_image, 'value')
block_direct_button.observe(update_image, 'value')
source_size_slider.observe(update_image, 'value')
structure_dropdown.observe(update_image, 'value')
signal_dropdown.observe(update_image, 'value')
power_scale_slider.observe(update_diffraction_pattern, 'value')

position_tool.observe(update_diffraction_pattern, 'indices')
position_tool.activate(canvas1)

canvas1.title = 'Image'
canvas2.title = 'Diffraction pattern'

update_image()
canvas1.adjust_limits_to_artists()

whitespace = widgets.HBox([])
whitespace.layout.height = '30px'

widgets.HBox([canvas1.widget, canvas2.widget, 
              widgets.VBox([whitespace, 
                            structure_dropdown, 
                            signal_dropdown, 
                            widgets.HBox([autoadjust_button, block_direct_button]), 
                            power_scale_slider,
                            source_size_slider, 
                            energy_spread_slider])])